# Roof detection in point clouds using DBSCAN clustering and RANSAC plane fitting


In [27]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [28]:
# Importing external packages
import warnings

import open3d as o3d

In [29]:
# Importing internal packages
from src.enums import Files
from src.features.point_cloud_handler import Handler
from src.features.pre_process import Reduce
from src.features.display import Visualize
from src.features.cluster import PointCloudDBSCAN
from src.features.ransac import RANSACPlaneFitting

In [30]:
# Package setup
warnings.filterwarnings('ignore')

## Initialization

Creating a point cloud object from a las/laz file.


In [31]:
cloud_compare_point_cloud: o3d.geometry.PointCloud = Handler.open(
    Files.OFF_GROUND_POINTS
)

Loaded 10434804 points from off_ground_points.las


## Preprocessing

### CFS and SOR

The first step in the preprocessing was using the cloth simulation filter (CFS) in Cloud Compare. This is done to separate the ground- and off-ground points. We are only interested in the off-ground points and will from now on only work with these points.
After this a statistical outlier removal (SOR) was done to remove the remaining noise.


### Voxel downsampling

The next step was to downsample the point cloud using voxel downsampling. This was done to reduce the number of points in the point cloud and to make the point cloud more uniform. The point cloud is also easier to work with due to the number of points are reduced. If is important to tune the paramaters in such a way that the semantic of the point cloud is maintaned and that there is enough points to be able to extract features from the point cloud.


In [32]:
downsampled_point_cloud: o3d.geometry.PointCloud = Reduce.voxel_downsample(
    cloud_compare_point_cloud
)

Original point count: 10434804
Downsampled point count: 2081090
Point cloud size reduced with 80.06%


## Detection

### Clustering

Using open3d DBSCAN clustering is used to find clusters in the point cloud. Some clusters contain few points and can be concidered noise. Thesee are relabled as noise and will not be considered in the RANSAC plane detection.


In [33]:
dbscan: PointCloudDBSCAN = PointCloudDBSCAN(
    point_cloud=downsampled_point_cloud
)

dbscan.cluster()

Point cloud has 352 clusters


### RANSAC plane fitting to detect roofs


In [34]:
ransac: 'RANSACPlaneFitting' = RANSACPlaneFitting(
    point_cloud=downsampled_point_cloud,
    dbscan=dbscan
)

In [35]:
ransac.run()

Running RANSAC Plane Fitting with the following parameters:
            - Plane Fitting Iteration: 1000
            - Sample Size: 3
            - Distance Threshold: 0.5
            - RANSAC Iteration Limit: 75
            - Standard Deviation Threshold: 14
            - Plane Inlier Ratio Limit: 0.07
            


RANSAC Plane Fitting for clusters: 100%|██████████| 352/352 [01:32<00:00,  3.79cluster/s]


In [36]:
Visualize.display(*ransac.roofs, ransac.point_cloud)
Handler.save(*ransac.roofs, ransac.point_cloud)

Saved point cloud to C:\Users\jatha\Code\NTNU\TBA4256-3D-Digital-Modelling\TBA4256-Assignment-1\data\output\20231022-144119.ply
